# TastyTrade API Authentication Tests

This notebook demonstrates various authentication flows for TastyTrade's API, with a focus on the remember-me token functionality.

## Setup

First, we'll import the necessary libraries and set up our environment.

In [ ]:
import requests
import json
import os
from datetime import datetime
from dotenv import load_dotenv
import pandas as pd
from IPython.display import display, Markdown

# Load environment variables
load_dotenv()

# Set up variables
base_url = os.getenv("TT_API_URL")
username = os.getenv("TT_USER")
password = os.getenv("TT_PASS")

print(f"Base URL: {base_url}")
print(f"Username: {username[:1]}" + "*" * (len(password)))
print(f"Password: {password[:1] + '*' * len(password) if password else 'Not set'}")

## Helper Functions

Let's define a few helper functions to make our authentication tests cleaner.

In [2]:
def display_request_response(auth_data, response, include_tokens=False):
    """Display formatted request and response data."""
    # Create a DataFrame for the request
    request_df = pd.DataFrame({
        'Key': list(auth_data.keys()),
        'Value': [str(v) if include_tokens else (v if k not in ['password', 'remember-token', 'login'] else v[:1] + '***') 
                 for k, v in auth_data.items()]
    })
    
    display(Markdown("### Request"))
    display(request_df)
    
    # Display the response
    display(Markdown(f"### Response (Status: {response.status_code})"))
    
    try:
        response_data = response.json()
        # Extract response data for display
        if 'data' in response_data and response.status_code < 400:
            data = response_data['data']
            
            # Create a flattened representation of important fields
            flat_data = {}
            
            # Handle user info
            if 'user' in data:
                for key, value in data['user'].items():
                    # Obfuscate email and username
                    if key in ['email', 'username']:
                        flat_data[f'user.{key}'] = value[:1] + '***'
                    else:
                        flat_data[f'user.{key}'] = value
            
            # Handle tokens and other direct fields
            for key, value in data.items():
                if key != 'user':
                    if key in ['session-token', 'remember-token'] and not include_tokens:
                        flat_data[key] = value[:10] + '...' if value else 'None'
                    else:
                        flat_data[key] = value
            
            # Create and display response DataFrame
            response_df = pd.DataFrame({
                'Field': list(flat_data.keys()),
                'Value': list(flat_data.values())
            })
            display(response_df)
            
            return data
        else:
            # For error responses
            display(Markdown(f"```json\n{json.dumps(response_data, indent=2)}\n```"))
            return None
    except Exception as e:
        display(Markdown(f"**Error parsing response:** {str(e)}\n\n```\n{response.text}\n```"))
        return None

def authenticate(auth_data, label="Authentication"):
    """Authenticate with the TastyTrade API and return structured tokens."""
    display(Markdown(f"## {label}"))
    
    try:
        # Create a copy of auth_data to avoid modifying the original
        display_auth_data = auth_data.copy()
        
        response = requests.post(
            f"{base_url}/sessions",
            headers={
                "Content-Type": "application/json", 
                "Accept": "application/json"
            },
            json=auth_data  # Use json parameter instead of data with json.dumps
        )
        
        # Display the formatted request and response
        response_data = display_request_response(display_auth_data, response)
        
        # Return structured token data if successful
        if response.status_code < 400 and response_data:
            return {
                "remember_token": response_data.get("remember-token"),
                "session_token": response_data.get("session-token"),
                "expiration": response_data.get("session-expiration"),
                "timestamp": datetime.now().isoformat()
            }
        return None
        
    except Exception as e:
        display(Markdown(f"**Error:** {str(e)}"))
        return None

## Test 1: Initial Authentication with Password

First, we'll authenticate using username and password. If successful, this will return both a session token and a remember token.

In [ ]:
# Test 1: Authenticate with username and password, requesting a remember token
auth_data = {
    "login": username, 
    "password": password, 
    "remember-me": True
}

token_data_1 = authenticate(auth_data, "Initial Authentication with Password")

# Store the tokens for later use
remember_token_1 = token_data_1.get("remember_token") if token_data_1 else None
session_token_1 = token_data_1.get("session_token") if token_data_1 else None

if remember_token_1:
    display(Markdown("**✓ Successfully obtained remember token**"))
else:
    display(Markdown("**✗ Failed to obtain remember token**"))

## Test 2: Using the Remember Token

Now, we'll try to authenticate using the remember token obtained in Test 1.

In [ ]:
# Test 2: Authenticate with the remember token
if remember_token_1:
    auth_data = {
        "login": username,
        "remember-token": remember_token_1,
        "remember-me": True
    }
    
    token_data_2 = authenticate(auth_data, "Authentication with Remember Token")
    
    # Store the new tokens
    remember_token_2 = token_data_2.get("remember_token") if token_data_2 else None
    session_token_2 = token_data_2.get("session_token") if token_data_2 else None
    
    if remember_token_2:
        display(Markdown("**✓ Successfully authenticated with remember token**"))
    else:
        display(Markdown("**✗ Failed to authenticate with remember token**"))
else:
    display(Markdown("**⚠️ Skipping this test because no remember token is available**"))

## Test 3: Testing if Remember Tokens are Single-Use

Next, we'll try to reuse the first remember token to see if it's still valid. If remember tokens are single-use, this should fail.

In [ ]:
# Test 3: Try to reuse the first remember token (should fail if tokens are single-use)
if remember_token_1 and remember_token_2:  # Only proceed if we got both tokens
    auth_data = {
        "login": username,
        "remember-token": remember_token_1,
        "remember-me": True
    }
    
    token_data_3 = authenticate(auth_data, "Reusing the First Remember Token")
    
    if token_data_3 and token_data_3.get("remember_token"):
        display(Markdown("**⚠️ First remember token was reusable (not single-use)**"))
    else:
        display(Markdown("**✓ First remember token was single-use (expected behavior)**"))
else:
    display(Markdown("**⚠️ Skipping this test because we don't have both remember tokens**"))

## Test 4: Testing if New Remember Token is Valid

Now, we'll verify that the new remember token obtained in Test 2 is valid.

In [ ]:
# Test 4: Use the second remember token (should succeed)
if remember_token_2:
    auth_data = {
        "login": username,
        "remember-token": remember_token_2,
        "remember-me": True
    }
    
    token_data_4 = authenticate(auth_data, "Using the Second Remember Token")
    
    remember_token_3 = token_data_4.get("remember_token") if token_data_4 else None
    
    if remember_token_3:
        display(Markdown("**✓ Successfully authenticated with second remember token**"))
    else:
        display(Markdown("**✗ Failed to authenticate with second remember token**"))
else:
    display(Markdown("**⚠️ Skipping this test because we don't have the second remember token**"))

## Test 5: Testing remember-me=False Flag

Finally, let's test what happens when we set remember-me to False. We should still be able to authenticate, but may not get a new remember token.

In [ ]:
# Test 5: Try with remember-me: False to see if it affects token generation
if remember_token_3:
    auth_data = {
        "login": username,
        "remember-token": remember_token_3,
        "remember-me": False  # Don't request a new remember token
    }
    
    token_data_5 = authenticate(auth_data, "Authentication with remember-me: False")
    
    remember_token_4 = token_data_5.get("remember_token") if token_data_5 else None
    
    if remember_token_4:
        display(Markdown("**⚠️ Got a new remember token even with remember-me: False**"))
    else:
        display(Markdown("**✓ No new remember token when remember-me: False (expected behavior)**"))
else:
    display(Markdown("**⚠️ Skipping this test because we don't have the third remember token**"))

## Bonus Test: Making an Authenticated Request

Let's try making an authenticated API request using the session token we obtained.

In [ ]:
# Get the most recent session token
session_token = None
for token_data in [token_data_5, token_data_4, token_data_3, token_data_2, token_data_1]:
    if token_data and token_data.get("session_token"):
        session_token = token_data.get("session_token")
        break

if session_token:
    display(Markdown("## Making an Authenticated Request"))
    try:
        # Request the customer profile as a test
        response = requests.get(
            f"{base_url}/customers/me",
            headers={
                "Authorization": session_token,
                "Accept": "application/json"
            }
        )

        display(Markdown(f"### Customer Profile Request (Status: {response.status_code})"))

        if response.status_code == 200:
            profile_data = response.json()

            # Create a more readable display of the user profile
            if 'data' in profile_data:
                user_data = profile_data['data']

                # Filter to just the most important fields
                important_fields = [
                    'username', 'email', 'first-name', 'last-name', 'phone-number',
                    'created-at', 'country', 'account-status'
                ]

                filtered_data = {}
                for field in important_fields:
                    if field in user_data:
                        # Obfuscate sensitive information
                        if field in ['username', 'email']:
                            first_letter = user_data[field][0]
                            filtered_data[field] = first_letter + "***"
                        elif field == 'first-name' and user_data[field]:
                            # Show only first letter of first name
                            first_letter = user_data[field][0]
                            filtered_data[field] = f"{first_letter}{'*' * (len(user_data[field]) - 1)}"
                        elif field == 'last-name' and user_data[field]:
                            # Show only first letter of last name
                            first_letter = user_data[field][0]
                            filtered_data[field] = f"{first_letter}{'*' * (len(user_data[field]) - 1)}"
                        else:
                            filtered_data[field] = user_data[field]

                # Display as a DataFrame
                profile_df = pd.DataFrame({
                    'Field': list(filtered_data.keys()),
                    'Value': list(filtered_data.values())
                })
                display(profile_df)

                display(Markdown("**✓ Successfully retrieved user profile**"))
            else:
                display(Markdown(f"```json\n{json.dumps(profile_data, indent=2)}\n```"))
        else:
            display(Markdown(f"**✗ Error retrieving profile: {response.status_code}**"))
            display(Markdown(f"```\n{response.text}\n```"))
    except Exception as e:
        display(Markdown(f"**Error making authenticated request:** {str(e)}"))
else:
    display(Markdown("**⚠️ Cannot make authenticated request because no session token is available**"))

## Summary

Let's summarize what we've learned about TastyTrade's authentication system:

1. Initial authentication uses username + password and can return both a session token and a remember token.
2. Remember tokens can be used for subsequent authentication without sending the password.
3. Remember tokens appear to be single-use - once used, they are invalidated.
4. Each successful authentication with a remember token issues a new remember token.
5. Setting `remember-me: false` may affect whether a new remember token is issued.
6. Session tokens are used for making authenticated requests to the API.